In [1]:
import os

In [2]:
%pwd

'e:\\Data Science\\ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Data Science\\ML-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/shubh-2291/ML_Project1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="shubh-2291"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bda8667039f829190ef0956963cf5992817e71f0"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from ML_Project.constants import *
from ML_Project.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/shubh-2291/ML_Project1.mlflow"
        )
        
        return model_evaluation_config
    

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlflow.models.signature import infer_signature

In [10]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]        
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        print("MLflow Tracking URI:", mlflow.get_tracking_uri())
        
        with mlflow.start_run() as run:
            predicted_qualities = model.predict(test_x)
            
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)
            
            signature = infer_signature(test_x, predicted_qualities)
            
            scores = {"rmse":rmse, "mae": mae, "r2": r2}
            save_json(path= Path(self.config.metric_file_name), data= scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2":r2})
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(sk_model= model, signature=signature, registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-02-24 10:47:27,283: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-24 10:47:27,312: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-24 10:47:27,340: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-24 10:47:27,344: INFO: common: created direcory at: artifacts]
[2025-02-24 10:47:27,347: INFO: common: created direcory at: artifacts/model_evaluation]
MLflow Tracking URI: https://dagshub.com/shubh-2291/ML_Project1.mlflow


2025/02/24 10:47:28 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}